In [1]:
#Working ok 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pydicom as pdcm
import os
import csv
import pandas as pd
import re

In [4]:
#GET PATHS AND LABELS
root_path  = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/"
PxList = np.sort(os.listdir(root_path))
csv_file_path = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/BP4DCTs_Lists_v3.csv"

fieldnames=['ID', '0%', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%']


In [6]:
#LOOK for patients already been analized
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    checked_list = df['ID'].tolist()
    print(len(checked_list),"Px already checked,out of",len(PxList))
else:
    checked_list = []

In [7]:
def CheckIfdIsBP(d,BPCollected_count,BP2Check_count,BP00,BP10,BP20,BP30,BP40,BP50,BP60,BP70,BP80,BP90,BP100):
    if not('ave' in d.lower()) and not('mip' in d.lower()):
        if ("S3 0%" in d or " 0% iMAR" in d or "Y 0%" in d or "T=0%" in d or re.search(r'100% (ex|Ex|EX)',d) or re.search(r'100%(ex|Ex|EX)',d)): 
            BP00.append(root+'/'+d+'/')
            BPCollected_count+=1
            BP2Check_count+=1
        if ("S3 10%" in d or "10% iMAR" in d or "Y 10%" in d or "T=10%" in d or re.search(r'[789]\d% (ex|Ex|EX)',d) or re.search(r'[789]\d%(ex|Ex|EX)',d)):
            BP10.append(root+'/'+d+'/')
            BPCollected_count+=1
            BP2Check_count+=1
        if ("S3 20%" in d or "20% iMAR" in d or "Y 20%" in d or "T=20%" in d or re.search(r'[456]\d% (ex|Ex|EX)',d) or re.search(r'[456]\d%(ex|Ex|EX)',d)): 
            BP20.append(root+'/'+d+'/')
            BPCollected_count+=1
        if ("S3 30%" in d or "30% iMAR" in d or "Y 30%" in d or "T=30%" in d): 
            BP30.append(root+'/'+d+'/')
            BPCollected_count+=1
        if ("S3 40%" in d or "40% iMAR" in d or "Y 40%" in d or "T=40%" in d or re.search(r'[123]\d% (ex|Ex|EX)',d) or re.search(r'[123]\d%(ex|Ex|EX)',d)): 
            BP40.append(root+'/'+d+'/')
            BPCollected_count+=1
        if ("S3 50%" in d or "50% iMAR" in d or "Y 50%" in d or "T=50%" in d or re.search(r' \d% (ex|Ex|EX)',d) or re.search(r' \d%(ex|Ex|EX)',d)): 
            BP50.append(root+'/'+d+'/')
            BPCollected_count+=1
            BP2Check_count+=1
        if ("S3 60%" in d or "60% iMAR" in d or "Y 60%" in d or "T=60%" in d or re.search(r' \d% (in|In|IN)',d) or re.search(r' \d%(in|In|IN)',d)): 
            BP60.append(root+'/'+d+'/')
            BPCollected_count+=1
            BP2Check_count+=1
        if ("S3 70%" in d or "70% iMAR" in d or "Y 70%" in d or "T=70%" in d or re.search(r'[123]\d% (in|In|IN)',d) or re.search(r'[123]\d%(in|In|IN)',d)): 
            BP70.append(root+'/'+d+'/')
            BPCollected_count+=1
        if ("S3 80%" in d or "90% iMAR" in d or "Y 80%" in d or "T=80%" in d or re.search(r'[456]\d% (in|In|IN)',d) or re.search(r'[456]\d%(in|In|IN)',d)): 
            BP80.append(root+'/'+d+'/')
            BPCollected_count+=1
        if ("S3 90%" in d or "90% iMAR" in d or "Y 90%" in d or "T=90%" in d or re.search(r'[789]\d% (in|In|IN)',d) or re.search(r'[789]\d%(in|In|IN)',d)): 
            BP90.append(root+'/'+d+'/')
            BPCollected_count+=1
            BP2Check_count+=1
        if ("S3 100%" in d or "100% iMAR" in d or "Y 100%" in d or "T=100%" in d or re.search(r'100% (in|In|IN)',d) or re.search(r'100%(in|In|IN)',d)): 
            BP100.append(root+'/'+d+'/')
            BPCollected_count+=1
            BP2Check_count+=1
    return BPCollected_count,BP2Check_count,BP00,BP10,BP20,BP30,BP40,BP50,BP60,BP70,BP80,BP90,BP100

In [8]:
with open(csv_file_path, 'w', newline='') as csvfile:
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if csvfile.tell() == 0:
        writer.writeheader()
    lessthan2BPpx=[]
    incompletePx = [] #No resplow
    completePx = []

    for Px in PxList:
        BP00,BP10,BP20,BP30,BP40,BP50,BP60,BP70,BP80,BP90,BP100 = ([],[],[],[],[],[],[],[],[],[],[])

        BPCollected_count=0
        BP2Check_count=0
        Tag = None

        for root, dirs, files in os.walk(root_path+Px, topdown=True):
            for d in dirs:
                BPCollected_count,BP2Check_count,BP00,BP10,BP20,BP30,BP40,BP50,BP60,BP70,BP80,BP90,BP100 = CheckIfdIsBP(d,BPCollected_count,BP2Check_count,BP00,BP10,BP20,BP30,BP40,BP50,BP60,BP70,BP80,BP90,BP100)
            if BPCollected_count ==0:
                for d in dirs:
                    if 'resplow' in d.lower():
                        files = os.listdir(root+'/'+d+'/')
                        slices = pdcm.read_file(os.path.join(root,d,files[0]))
                        newd = (d+'_'+slices.ImageComments).replace(' ','')
                        print('resplow: ',d,'to',newd)
                        BPCollected_count,BP2Check_count,BP00,BP10,BP20,BP30,BP40,BP50,BP60,BP70,BP80,BP90,BP100 = CheckIfdIsBP(newd,BPCollected_count,BP2Check_count,BP00,BP10,BP20,BP30,BP40,BP50,BP60,BP70,BP80,BP90,BP100)
        if BP2Check_count>=2:
            completePx.append(Px)
            Tag = "Complete"
        else:
            lessthan2BPpx.append(Px)
            Tag = "Unusuable"

        if BPCollected_count<8:
            if 'RespLow' in dirs:
                incompletePx.append(Px)
                Tag = "Incomplete"


        #print("Px", Px,Tag, len(BP00), len(BP10), len(BP20), len(BP30), len(BP40), len(BP50), len(BP60), len(BP70),len(BP80), len(BP90), len(BP100))

        max_length = max(len(BP00), len(BP10), len(BP20), len(BP30), len(BP40), len(BP50), len(BP60), len(BP70),len(BP80), len(BP90), len(BP100))

        BP_lists = [BP00, BP10, BP20, BP30, BP40, BP50, BP60, BP70, BP80, BP90, BP100]
        for i in range(len(BP_lists)):
            BP_lists[i] += [0] * (max_length - len(BP_lists[i]))

        writer.writerow({'ID': Px, '0%': BP00, '10%': BP10, '20%': BP20, '30%': BP30, '40%': BP40, '50%': BP50,'60%': BP60, '70%': BP70, '80%': BP80, '90%': BP90, '100%': BP100})
print("Complete",len(completePx),"Unsusable",len(lessthan2BPpx),"Incomplete",len(incompletePx))
            
            

resplow:  RespLow  3.0  B30s to RespLow3.0B30s_17rpm,45%In,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_21rpm,97%In,500ms
resplow:  tmp  RespLow  RTD to tmpRespLowRTD_
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_15rpm,95%In,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_In.92%
5rpm,92%In,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_17rpm,40%In,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_18rpm,35%In,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_10rpm,5%Ex,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_12rpm,100%In,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_4rpm,45%In,500ms
resplow:  RespLow  3.0  B30s to RespLow3.0B30s_30rpm,85%In,500ms
resplow:  RespLow  3.0 4D  B30s to RespLow3.04DB30s_22rpm,12%Ex,500ms
Complete 1124 Unsusable 128 Incomplete 0


In [9]:
#Complete 1113 Unsusable 1998 Incomplete 0
#Complete 1124 Unsusable 128 Incomplete 0



FileNotFoundError: [WinError 3] The system cannot find the path specified: '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9970729\\X5 THORAX\\T=60%,PR=57% - 67%,AR()=74 - 95/T=60%,PR=57% - 67%,AR()=74 - 95/'